<a href="https://colab.research.google.com/github/Thinley-bot/Translator/blob/main/Dzo2Env3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the first version of neural machine translation using the encoder decoder and keras libraries. ⚓

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [3]:
import string
import re
from numpy import array, argmax, random, take
import numpy as np
from numpy.random import shuffle
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences
from keras.models import load_model
from keras import optimizers
from keras.layers import TimeDistributed
import matplotlib.pyplot as plt
from pickle import dump
from unicodedata import normalize
from keras.models import load_model
import unicodedata
import pickle as pkl

In [4]:
# Defining the path to the raw data set
fileurl = '/content/drive/MyDrive/Final Year Project/eng-dzo.txt'

In [5]:
# function to read raw text file
def read_text(filename):
    # open the file
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    
    # Split the text into individual lines
    lines = text.strip().split('\n')
    # Splitting each line based on tab spaces and creating a list
    lines = [line.split('\t') for line in lines]

    file.close()
    return array(lines)

In [6]:
# Reading the data using the function
mtData = read_text(fileurl)
# Taking only 50000 rows of data
mtData = mtData[:50000,:2]
print(mtData.shape)
mtData[0:10]

(50000, 2)


array([['They smiled at one another.',
        'ཁོང་ ཚུ་ གཅིག་གིས་གཅིག་ལུ་ བལྟ་ སྟེ་ དགའ་འཛུམ་ སྟོན་ ནུག །'],
       ['They want to do this right.',
        'ཁོང་ ཚུ་ གིས་ ཨ་ནཱི་ དྲང་པོ་ སྦེ་ འབད་ ནི་ མས །'],
       ['They want to do this right away.',
        'ཁོང་ ཚུ་ གིས་ ཨ་ནཱི་ འདི་འཕྲོ་ལས་ འབད་ ནི་ མས །'],
       ['They watched me in silence.',
        'ཁོང་ ཚུ་ གིས་ ང་ ལུ་ ཁུ་སིམ་སི་ སྦེ་ བལྟ་ སྡོད་ ནུག །'],
       ['They water the fruit trees.',
        'ཁོང་ ཚུ་ གིས་ ཤིང་འབྲས་ ཀྱི་ ཤིང་ ནང་ ཆུ་ བླུགས་ ནུག །'],
       ['They were acting strangely.',
        'ཁོང་ ཚུ་ གིས་ ཧ་གོ་ མ་ ཚུགས་ པའི་ རྣམ་འགྱུར་ སྟོན་ མས །'],
       ['They were born in Thailand.',
        'ཁོང་ ཚུ་ ཐའི་ལེནཌ་ ལུ་ སྐྱེས་ ནུག །'],
       ['They were tired of waiting.',
        'ཁོང་ ཚུ་ སྒུག་ སྟེ་ ར་ སྡོད་ དེ་ འུ་སྡུག་ ནུག །'],
       ['They were tired of watching.',
        'ཁོང་ ཚུ་ བལྟ་ སྟེ་ ར་ སྡོད་ དེ་ འུ་སྡུག་ ནུག །'],
       ['They were tired of walking.',
        'ཁོང་ ཚུ་ ལམ་འགྱོ་ སྟེ་ ར་ སྡོད་ དེ་ 

In [7]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def preprocess_english(s):
    s=str(s)
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([,.!?])", r" \1 ", s)
    s = re.sub(r"[^a-zA-Z,.!?]+", r" ", s)
    #replace the extra space by single space
    s = re.sub(r"\s+", r" ", s)
    return s
def preprocess_dzongkha(s):
  s=str(s)
  s = re.sub(r'་\s*[& ]', ' ',s)
  s=re.sub(r"\s*།\s*$", '', s)
  s=re.sub(r'\s#\s',' ',s)
  return s

In [8]:
# Cleaning the document from all unwanted characters

def cleanDocs(lines):
    cleanArray = list()
    for pair in lines:
        cleanDocs = list()
        # for line in pair:
       
            # print(line)
        for i, line in enumerate(pair):
            if i==0:
                line=preprocess_english(line)
                # print(line)
            else:
                line=preprocess_dzongkha(line)
            # line = normalize('NFD', line).encode()
            # line = line.decode('UTF-8')
            line = line.split()
            # line = [word.lower() for word in line]
            # line = [word.translate(table) for word in line]
            # line = [re_print.sub('', word) for word in line]
            cleanDocs.append(' '.join(line))
            # print(clean_pair)
            
        cleanArray.append(cleanDocs)
    return array(cleanArray)

In [9]:
# Cleaning the sentences
cleanMtDocs = cleanDocs(mtData)
cleanMtDocs[0:10]

array([['they smiled at one another .',
        'ཁོང ཚུ གཅིག་གིས་གཅིག་ལུ བལྟ སྟེ དགའ་འཛུམ སྟོན ནུག'],
       ['they want to do this right .',
        'ཁོང ཚུ གིས ཨ་ནཱི དྲང་པོ སྦེ འབད ནི མས'],
       ['they want to do this right away .',
        'ཁོང ཚུ གིས ཨ་ནཱི འདི་འཕྲོ་ལས འབད ནི མས'],
       ['they watched me in silence .',
        'ཁོང ཚུ གིས ང ལུ ཁུ་སིམ་སི སྦེ བལྟ སྡོད ནུག'],
       ['they water the fruit trees .',
        'ཁོང ཚུ གིས ཤིང་འབྲས ཀྱི ཤིང ནང ཆུ བླུགས ནུག'],
       ['they were acting strangely .',
        'ཁོང ཚུ གིས ཧ་གོ མ ཚུགས པའི རྣམ་འགྱུར སྟོན མས'],
       ['they were born in thailand .', 'ཁོང ཚུ ཐའི་ལེནཌ ལུ སྐྱེས ནུག'],
       ['they were tired of waiting .',
        'ཁོང ཚུ སྒུག སྟེ ར སྡོད དེ འུ་སྡུག ནུག'],
       ['they were tired of watching .',
        'ཁོང ཚུ བལྟ སྟེ ར སྡོད དེ འུ་སྡུག ནུག'],
       ['they were tired of walking .',
        'ཁོང ཚུ ལམ་འགྱོ སྟེ ར སྡོད དེ འུ་སྡུག ནུག']], dtype='<U156')

In [10]:
# The dimensions of the data set
len(cleanMtDocs)
print(cleanMtDocs.shape)

(50000, 2)


In [11]:
# Shuffling the data
shuffle(cleanMtDocs)
cleanMtDocs[0:10]

array([['hand me the remote .', 'རྒྱང་རིང འདི ང ལུ སྤྲོད གནང'],
       ['well done !', 'ལེགས་ཤོམ བཏང ནུག མས'],
       ['i m eating lunch .', 'ང ཉི་མ་གི་ལྟོ ཟ དོ'],
       ['karma is sweating .', 'ཀརྨ རྔུལ་ནག འཐོན དེས'],
       ['i met a friend .', 'ང ཆ་རོགས ཆིག དང ཕྱད ཅི'],
       ['this makes no sense to me .',
        'འདི་གིས ང ལུ དོན་དག ག་ནི ཡང མི བཟོ བས'],
       ['i ll show you my room .', 'ང གིས ཁྱོད ལུ ངེའི ཁང མི སྟོན གེ'],
       ['get down from there .', 'ཕར ལས མར འབབ'],
       ['it rained for a week .', 'བདུན་ཕྲག གཅིག ར ཆརཔ རྐྱབ ཅི'],
       ['they have no more wine .', 'ཁོང ལུ ད ཝའིན མིན་འདུག']],
      dtype='<U156')

In [12]:
#save the clean data as pickle file
def save_data(sentences, filename):
    pkl.dump(sentences, open(filename, 'wb'))
    print(f'Saved: {filename}')

In [13]:
save_data(cleanMtDocs,'english-dzongkha.pkl')

Saved: english-dzongkha.pkl


In [14]:
# Checking the cleaned data
for i in range(100):
    print('%s => %s' % (cleanMtDocs[i, 0], cleanMtDocs[i, 1]))

hand me the remote . => རྒྱང་རིང འདི ང ལུ སྤྲོད གནང
well done ! => ལེགས་ཤོམ བཏང ནུག མས
i m eating lunch . => ང ཉི་མ་གི་ལྟོ ཟ དོ
karma is sweating . => ཀརྨ རྔུལ་ནག འཐོན དེས
i met a friend . => ང ཆ་རོགས ཆིག དང ཕྱད ཅི
this makes no sense to me . => འདི་གིས ང ལུ དོན་དག ག་ནི ཡང མི བཟོ བས
i ll show you my room . => ང གིས ཁྱོད ལུ ངེའི ཁང མི སྟོན གེ
get down from there . => ཕར ལས མར འབབ
it rained for a week . => བདུན་ཕྲག གཅིག ར ཆརཔ རྐྱབ ཅི
they have no more wine . => ཁོང ལུ ད ཝའིན མིན་འདུག
what is an off peak online only ticket ? => ཡོངས་འབྲེལ ཐོག རྐྱངམ་གཅིག ཡོད མི ཤོག་འཛིན འདི ག་ཅི སྨོ
i would like an air conditioned room . => ང གིས སྙོམས་རླུང ཡོད པའི ཁང་མིག ཅིག དགའ ནི མས
it s useless to try . => འབད་རྩོལ བསྐྱེད དེ ཁུངས མེད
you can t blame me . => ཁྱོད ང ལུ རྫུན་ཁ བཀལ མི ཆོག
i m retired . => ང དགོངས་ཞུ འབད ཡི
how s the family ? => བཟའ་ཚང འདི ག་དེ་སྦེ ཡོད
pick a number . => ཨང་གྲངས ཅིག འཐུ
i still have no idea what s going on . => ག་ཅི འབདཝ ཨིན་ན ད་ལྟོ ཡང ང ལུ གནས་ཚུལ ཅིག མེད
don t wait for me

Starting the neural translation

In [15]:
# Creating the tokenizers
# Function for creating tokenizers
def createTokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
# Create English Tokenizer
eng_tokenizer = createTokenizer(cleanMtDocs[:,0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
print('Length of english vocabulary',eng_vocab_size)

Length of english vocabulary 13163


In [ ]:
# Listing the first 10 items of the English tokenizer
list(eng_tokenizer.word_index.items())[0:10]

[('i', 1),
 ('the', 2),
 ('you', 3),
 ('a', 4),
 ('is', 5),
 ('to', 6),
 ('it', 7),
 ('t', 8),
 ('s', 9),
 ('karma', 10)]

In [ ]:
# Create Dzongkha tokenizer
dzo_tokenizer = createTokenizer(cleanMtDocs[:,1])
# Defining Dzongkha Vocabulary
dzo_vocab_size = len(dzo_tokenizer.word_index) + 1
print(dzo_vocab_size)

16958


In [ ]:
# Listing the first 10 items of the English tokenizer
list(dzo_tokenizer.word_index.items())[0:20]

[('ང', 1),
 ('འདི', 2),
 ('གིས', 3),
 ('ལུ', 4),
 ('ཁྱོད', 5),
 ('ཨིན', 6),
 ('ཡི', 7),
 ('ཅིག', 8),
 ('མི', 9),
 ('འབད', 10),
 ('ནི', 11),
 ('གི', 12),
 ('མས', 13),
 ('ཁོ', 14),
 ('ག', 15),
 ('ང་བཅས', 16),
 ('མ', 17),
 ('ཀྱིས', 18),
 ('ནུག', 19),
 ('ཡོད', 20)]

In [ ]:
# Create an empty list to store all english sentence lenghts
len_english = []
# Getting the length of all the English sentences
[len_english.append(len(line.split())) for line in cleanMtDocs[:,0]]
len_english[0:10]

[6, 5, 7, 5, 7, 6, 6, 5, 6, 4]

In [ ]:
len_Dzongkha = []
# Getting the length of all the Dzongkha sentences
[len_Dzongkha.append(len(line.split())) for line in cleanMtDocs[:,1]]
len_Dzongkha[0:10]

[5, 3, 12, 4, 8, 4, 6, 7, 4, 4]

Finding the optimimum sequence lengths.

In [ ]:
# Find the quantile length
engLength = np.quantile(len_english, .975)
engLength

11.0

In [ ]:
# Find the quantile length
dzoLength = np.quantile(len_Dzongkha, .975)
dzoLength

13.0

Encoding the sequences

In this phase we will encode each of the sentences as integers in a sequence. Another task which needs to be done is to ensure that the lengths are standard. This is the reason we calcualated the maximum length of each sequence. We get the lengths standard by zero padding the sequences.

In [ ]:
# Function for encoding and padding sequences

def encode_sequences(tokenizer,length, lines):
    # Sequences as integers
    X = tokenizer.texts_to_sequences(lines)
    # Padding the sentences with 0
    X = pad_sequences(X,maxlen=length,padding='post')
    return X

In [ ]:
# Preparing the train and test splits
from sklearn.model_selection import train_test_split
# split data into train and test set
train, test = train_test_split(cleanMtDocs, test_size=0.1, random_state = 123)
print(train.shape)
print(test.shape)

(45000, 2)
(5000, 2)


In [ ]:
# Creating the X variable for both train and test sets
trainX = encode_sequences(dzo_tokenizer,int(dzoLength),train[:,1])
testX = encode_sequences(dzo_tokenizer,int(dzoLength),test[:,1])
print(trainX.shape)
print(testX.shape)

(45000, 13)
(5000, 13)


In [ ]:
# Displaying first 5 rows of the traininig set
trainX[0:5]

array([[   1,    3,    2,  631,  250,    7,    0,    0,    0,    0,    0,
           0,    0],
       [ 482,   70, 5865,   31,  162,  470,    0,    0,    0,    0,    0,
           0,    0],
       [5612,    2,  168,   56,  385,   26,   11,  107,    0,    0,    0,
           0,    0],
       [   1,    3,   43,   26,  200,  145,    7,    0,    0,    0,    0,
           0,    0],
       [   9,   21,   92,    5,   63,   10,    9,    2,  467,   10,   62,
          40,    0]], dtype=int32)

In [ ]:
# Creating the Y variable both train and test
trainY = encode_sequences(eng_tokenizer,int(engLength),train[:,0])
testY = encode_sequences(eng_tokenizer,int(engLength),test[:,0])
print(trainY.shape)
print(testY.shape)

(45000, 11)
(5000, 11)


Model Architecture

In [ ]:
def defineModel(src_vocab,tar_vocab,src_timesteps,tar_timesteps,n_units):
    model = Sequential()
    model.add(Embedding(src_vocab,n_units,input_length=src_timesteps,mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units,return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab,activation='softmax')))
    # Compiling the model
    model.compile(optimizer = 'adam',loss='sparse_categorical_crossentropy')
    # Summarising the model
    model.summary()
    
    return model

In [ ]:
model = defineModel(dzo_vocab_size,eng_vocab_size,int(dzoLength),int(engLength),256)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 256)           4341248   
                                                                 
 lstm (LSTM)                 (None, 256)               525312    
                                                                 
 repeat_vector (RepeatVector  (None, 11, 256)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 11, 256)           525312    
                                                                 
 time_distributed (TimeDistr  (None, 11, 13163)        3382891   
 ibuted)                                                         
                                                                 
Total params: 8,774,763
Trainable params: 8,774,763
Non-

In [ ]:
# Fitting the model
checkpoint = ModelCheckpoint('model1.h5',monitor='val_loss',verbose=1,save_best_only=True,mode='min')
model.fit(trainX,trainY,epochs=50,batch_size=10,validation_data=(testX,testY),callbacks=[checkpoint],verbose=2)

Epoch 1/50

Epoch 1: val_loss improved from inf to 2.88874, saving model to model1.h5
4500/4500 - 61s - loss: 3.2507 - val_loss: 2.8887 - 61s/epoch - 14ms/step
Epoch 2/50

Epoch 2: val_loss improved from 2.88874 to 2.51237, saving model to model1.h5
4500/4500 - 42s - loss: 2.6194 - val_loss: 2.5124 - 42s/epoch - 9ms/step
Epoch 3/50

Epoch 3: val_loss improved from 2.51237 to 2.34414, saving model to model1.h5
4500/4500 - 42s - loss: 2.2400 - val_loss: 2.3441 - 42s/epoch - 9ms/step
Epoch 4/50

Epoch 4: val_loss improved from 2.34414 to 2.25961, saving model to model1.h5
4500/4500 - 43s - loss: 1.9606 - val_loss: 2.2596 - 43s/epoch - 10ms/step
Epoch 5/50

Epoch 5: val_loss improved from 2.25961 to 2.21625, saving model to model1.h5
4500/4500 - 41s - loss: 1.7204 - val_loss: 2.2163 - 41s/epoch - 9ms/step
Epoch 6/50

Epoch 6: val_loss improved from 2.21625 to 2.20381, saving model to model1.h5
4500/4500 - 41s - loss: 1.5096 - val_loss: 2.2038 - 41s/epoch - 9ms/step
Epoch 7/50

Epoch 7: val

Inferencing and evaluation

In [ ]:
# loading the model from the best model saved
model = load_model('model1.h5')

In [ ]:
# Generating the predictions
prediction = model.predict(testX,verbose=0)
prediction.shape

(5000, 11, 13163)

In [ ]:
#Getting the prediction index along the last axis ( Vocabulary size axis)
predIndex = [argmax(vector,axis = -1) for vector in prediction]
predIndex[0:3]

[array([ 1, 25,  8, 41, 44,  1, 47,  0,  0,  0,  0]),
 array([  1,  23,   4,  61, 177,   0,   0,   0,   0,   0,   0]),
 array([ 2,  5, 16,  6,  6,  6,  0,  0,  0,  0,  0])]

In [ ]:
# Creating the reverse dictionary
reverse_eng = eng_tokenizer.index_word

In [ ]:
# Converting the tokens to a sentence
preds = []
for pred in predIndex[0]:
  if pred == 0:
        continue 
  preds.append(reverse_eng[pred])  
print(' '.join(preds))

i don t know how i go


In [ ]:
# Looking at the target sentence
preds = []
for pred in testY[0]:
  if pred == 0:
        continue 
  preds.append(reverse_eng[pred])  
print(' '.join(preds))

i don t know how far


In [ ]:
# Creating a function for converting sequences
def Convertsequence(tokenizer,source):
    target = list()
    reverse_eng = tokenizer.index_word
    for i in source:
        if i == 0:
            continue
        target.append(reverse_eng[int(i)])
    return ' '.join(target)

In [ ]:
# Function to generate predictions from source data
def generatePredictions(model,tokenizer,data):
    prediction = model.predict(data,verbose=0)
    AllPreds = []
    for i in range(len(prediction)):
        predIndex = [argmax(prediction[i, :, :], axis=-1)][0]
        target = Convertsequence(tokenizer,predIndex)
        AllPreds.append(target)
    return AllPreds

In [ ]:
# Generate predictions
predSent = generatePredictions(model,eng_tokenizer,testX[0:20,:])

In [ ]:
for i in range(len(testY[0:20])):
    targetY = Convertsequence(eng_tokenizer,testY[i:i+1][0])
    print("Original sentence : {} :: Prediction : {}".format([targetY],[predSent[i]]))

Original sentence : ['i don t know how far'] :: Prediction : ['i don t know how i go']
Original sentence : ['i m a nice guy'] :: Prediction : ['i m a good man']
Original sentence : ['the songs are divided for easy consult'] :: Prediction : ['the is was to to to']
Original sentence : ['i had a breakdown'] :: Prediction : ['i was dumbfounded']
Original sentence : ['he likes animals'] :: Prediction : ['he likes animals']
Original sentence : ['i forgot you were listening'] :: Prediction : ['i forgot you you retired']
Original sentence : ['are you ready for halloween'] :: Prediction : ['are you ready to to']
Original sentence : ['all of our calls are free and confidential'] :: Prediction : ['our our our our our our']
Original sentence : ['i ll get the check'] :: Prediction : ['i ll pay the']
Original sentence : ['i ll destroy it'] :: Prediction : ['i ll arrange it']
Original sentence : ['i laughed at his joke'] :: Prediction : ['i forgave his his']
Original sentence : ['use this'] :: Predic

Predicting on your own sentences

In [ ]:
def cleanInput(lines):
    cleanSent = []
    cleanDocs = list()
    for docs in lines.split():
        line=str(docs)
        line = re.sub(r'་\s*[& ]', ' ',line)
        line=re.sub(r"\s*།\s*$", '', line)
        line=re.sub(r'\s#\s',' ',line)
        cleanDocs.append(line)
    cleanSent.append(' '.join(cleanDocs))
    return array(cleanSent)

In [ ]:
# Trying different input sentences
inputSentence = 'གྲུ་ འདི་ ཆུ་ ནང་ ཐིམ་ དེས །'

In [ ]:
# Clean the input sentence
cleanText = cleanInput(inputSentence)
cleanText

array(['གྲུ་ འདི་ ཆུ་ ནང་ ཐིམ་ དེས '], dtype='<U27')

In [ ]:
# Encode the inputsentence as sequence of integers
seq1 = encode_sequences(dzo_tokenizer,int(dzoLength),cleanText)
seq1

array([[3592,   83,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]], dtype=int32)

In [ ]:
# Generate the prediction
predSent = generatePredictions(model,eng_tokenizer,seq1)

print("Original sentence : {} :: Prediction : {}".format([cleanText[0]],predSent))

Original sentence : ['གྲུ་ འདི་ ཆུ་ ནང་ ཐིམ་ དེས '] :: Prediction : ['this is a']


In [ ]:
inputSentence1 ='བྱི་ལི་ འདི་ ཉལ་ ནུག །' 
inputSentence2 ='འདི་ མོ་ གིས་ ཤེས །' 

for sentence in [inputSentence1,inputSentence2]:
  cleanText = cleanInput(sentence)
  seq1 = encode_sequences(dzo_tokenizer,int(dzoLength),cleanText)
  # Generate the prediction
  predSent = generatePredictions(model,eng_tokenizer,seq1)

  print("Original sentence : {} :: Prediction : {}".format([cleanText[0]],predSent))

Original sentence : ['བྱི་ལི་ འདི་ ཉལ་ ནུག '] :: Prediction : ['this is a']
Original sentence : ['འདི་ མོ་ གིས་ ཤེས '] :: Prediction : ['she knows know know']


In [ ]:
### Saving the tokenizers and other variables as pickle files
save_data(eng_tokenizer,'eng_tokenizer.pkl')
save_data(eng_vocab_size,'eng_vocab_size.pkl')
save_data(dzo_tokenizer,'dzo_tokenizer.pkl')
save_data(dzo_vocab_size,'dzo_vocab_size.pkl')
save_data(trainX,'trainX.pkl')
save_data(trainY,'trainY.pkl')
save_data(testX,'testX.pkl')
save_data(testY,'testY.pkl')
save_data(engLength,'len_english.pkl')
save_data(dzoLength,'len_Dzongkha.pkl')

Saved: eng_tokenizer.pkl
Saved: eng_vocab_size.pkl
Saved: dzo_tokenizer.pkl
Saved: dzo_vocab_size.pkl
Saved: trainX.pkl
Saved: trainY.pkl
Saved: testX.pkl
Saved: testY.pkl
Saved: len_english.pkl
Saved: len_Dzongkha.pkl
